# 000 Forecasting Bot

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IFP import IFP, list_questions

In [3]:
ifps = list_questions()['results']

In [4]:
today_ids = list(sorted([x['id'] for x in ifps])) if ifps else [27238, 27239, 27240, 27241]
today_ids

[27720, 27721, 27722, 27723, 27732, 27733]

In [5]:
ifps = {id: IFP(id) for id in today_ids}

In [6]:
from Forecaster import Forecaster

In [7]:
fcst = Forecaster(ifps)

In [8]:
fcst.fit()

MODEL DOMAIN
CORRELATOR
|Jensen Huang's Forbes Real-Time Billionaires Ranking on September 30, 2024|27720|Will Jensen Huang be in the top 10 of the Forbes Real-Time Billionaires List on September 30, 2024?|
|Plug Power's Bankruptcy Status before October 1, 2024|27721|Will Plug Power file for bankruptcy before October 1, 2024?|
|Nvidia Layoffs Report by Tech Crunch before October 1, 2024|27722|Before October 1, 2024, will Tech Crunch report new layoffs at Nvidia?|
|2024 MTV VMAs - Best Rock Award for Coldplay|27723|Will "Feelslikeimfallinginlove" by Coldplay win the award for Best Rock at the 2024 MTV VMAs?|
|Chinese Military Aircraft Violation of Japanese Airspace before October 1, 2024|27732|Will another Chinese military aircraft violate Japanese territorial airspace before October 1, 2024?|
|US Presidential Debate between Donald Trump and Kamala Harris on September 10, 2024|27733|Will there be a US presidential debate between Donald Trump and Kamala Harris held on September 10, 2024?

In [ ]:
fcst.report()

In [ ]:
fcst.upload()